In [ ]:
import numpy as np

In [ ]:
def get_path_to_substation(feeder, node, node_index_map):
    #returns list of edges between node and substation
    #node = node name as string
    graph = feeder.network
    node_name_path = []
    node_index_path = []
    current_node = node
    
    for i in range(depths[node]):
        pred_node = graph.predecessors(current_node)
        node_name_path += [(current_node, pred_node)]
        current_node = pred_node
    for edge_name in node_name_path:
        edge_index = (node_index_map[edge_name[0]], node_index_map[edge_name[1]])
        node_index_path += [edge_index]
    return node_index_path

def get_total_impedance_from_substation(feeder, node_name):
    node_name_full = "bus_" + node_name
    #Notice that since this is a tree, any node will only have at most one predecessor
    total_impedance = {'Phase 1' : 0.0, 'Phase 2' : 0.0, 'Phase 3' : 0.0}
    
    current_node = node_name_full
    pred_list = None
    
    try:
        pred_list = list(feeder.network.predecessors(node_name_full))
    except nx.NetworkXError:
        print("Bus with name " + current_node + " does not exist in the feeder.")
        return 0
    
    iter_depth = depths[current_node]
    
    for _ in range(iter_depth):
        #pred_list[0] is the parent node
        impedance = feeder.network.get_edge_data(pred_list[0], current_node, default=None)['connector']
        #print("Type is " + str(type(impedance)))
        if impedance == None:
            print("WARNING: No connection between nodes " + str(pred_list[0]) + " and " + str(current_node) + ".")
            return 0
        else:
            imp_dict = impedance.Z if isinstance(impedance, line) else np.zeros((3,3))
            
            total_impedance['Phase 1'] += imp_dict[0][0]
            total_impedance['Phase 2'] += imp_dict[1][1]
            total_impedance['Phase 3'] += imp_dict[2][2]
            
            current_node = pred_list[0]
            pred_list = list(feeder.network.predecessors(current_node))
    
    return total_impedance


def get_total_impedance_between_two_buses(feeder, node_name_1, node_name_2):
    bus_1 = "bus_" + node_name_1
    bus_2 = "bus_" + node_name_2
    
    total_impedance = {'Phase 1' : 0.0, 'Phase 2' : 0.0, 'Phase 3' : 0.0}
    
    depth_1 = 0
    depth_2 = 0
    
    try:
        depth_1 = depths[bus_1]
        depth_2 = depths[bus_2]
    except KeyError:
        print("Either the first bus, " + bus_1 + ", or the second bus, " + bus_2 + " is not a valid bus in the feeder.")
        return 0
    
    depth_dif = abs(depth_1 - depth_2)
    
    max_depth_bus = bus_1 if depth_1 > depth_2 else bus_2
    min_depth_bus = bus_1 if max_depth_bus == bus_2 else bus_2
    
    pred_list_max = list(feeder.network.predecessors(max_depth_bus))
    pred_list_min = list(feeder.network.predecessors(min_depth_bus))
    
    for i in range(depth_dif):
        
        impedance = feeder.network.get_edge_data(pred_list_max[0], max_depth_bus, default=None)['connector']
        if impedance == None:
            print("WARNING: No connection between nodes " + str(pred_list_max[0]) + " and " + str(max_depth_bus) + ".")
            return 0
        else:
            imp_dict = impedance.Z if isinstance(impedance, line) else np.zeros((3,3))
            
            total_impedance['Phase 1'] += imp_dict[0][0]
            total_impedance['Phase 2'] += imp_dict[1][1]
            total_impedance['Phase 3'] += imp_dict[2][2]
            
            #Case of where we the two buses are directly linked by purely upstream connections, allowing us to
            #terminate our calculations earlier
            if pred_list_max[0] == min_depth_bus:
                print("Iterated " + str(i+1) + " times to get direct upstream connection total impedance.")
                return total_impedance
            
            max_depth_bus = pred_list_max[0]
            pred_list_max = list(feeder.network.predecessors(max_depth_bus))
            
    assert(depths[max_depth_bus] == depths[min_depth_bus])
    
    print("Iterated " + str(depth_dif) + " times to reach equal depths.")
    
    common_parent = pred_list_max[0] == pred_list_min[0]
    
    count_get_to_common = 0
    
    #Here, we simultaneously shift both buses (after the max depth bus has been shifted to be of equal depth to the min
    #depth bus) to a point where the parent bus is shared
    while not common_parent:
        count_get_to_common += 1
        
        impedance_bus_min = feeder.network.get_edge_data(pred_list_min[0], min_depth_bus, default=None)['connector']
        if impedance_bus_min == None:
            print("WARNING: No connection between nodes " + str(pred_list_min[0]) + " and " + str(min_depth_bus) + ".")
            return 0
        else:
            imp_dict_min = impedance_bus_min.Z if isinstance(impedance_bus_min, line) else np.zeros((3,3))
            
            total_impedance['Phase 1'] += imp_dict_min[0][0]
            total_impedance['Phase 2'] += imp_dict_min[1][1]
            total_impedance['Phase 3'] += imp_dict_min[2][2]
            
            min_depth_bus = pred_list_min[0]
            pred_list_min = list(feeder.network.predecessors(min_depth_bus))
            
        impedance_bus_max = feeder.network.get_edge_data(pred_list_max[0], max_depth_bus, default=None)['connector']
        if impedance_bus_max == None:
            print("WARNING: No connection between nodes " + str(pred_list_max[0]) + " and " + str(max_depth_bus) + ".")
            return 0
        else:
            imp_dict_max = impedance_bus_max.Z if isinstance(impedance_bus_max, line) else np.zeros((3,3))
            
            total_impedance['Phase 1'] += imp_dict_max[0][0]
            total_impedance['Phase 2'] += imp_dict_max[1][1]
            total_impedance['Phase 3'] += imp_dict_max[2][2]
            
            max_depth_bus = pred_list_max[0]
            pred_list_max = list(feeder.network.predecessors(max_depth_bus))
            
        common_parent = pred_list_max[0] == pred_list_min[0]
    
    print("Total iterations to get to common parent is " + str(count_get_to_common))
    print("Common parent is " + str(pred_list_max[0]))
    
    #Need to iterate one more time to account for "joining" node
    impedance_bus_min = feeder.network.get_edge_data(pred_list_min[0], min_depth_bus, default=None)['connector']
    if impedance_bus_min == None:
        print("WARNING: No connection between nodes " + str(pred_list_min[0]) + " and " + str(min_depth_bus) + ".")
        return 0
    else:
        imp_dict_min = impedance_bus_min.Z if isinstance(impedance_bus_min, line) else np.zeros((3,3))

        total_impedance['Phase 1'] += imp_dict_min[0][0]
        total_impedance['Phase 2'] += imp_dict_min[1][1]
        total_impedance['Phase 3'] += imp_dict_min[2][2]

    impedance_bus_max = feeder.network.get_edge_data(pred_list_max[0], max_depth_bus, default=None)['connector']
    if impedance_bus_max == None:
        print("WARNING: No connection between nodes " + str(pred_list_max[0]) + " and " + str(max_depth_bus) + ".")
        return 0
    else:
        imp_dict_max = impedance_bus_max.Z if isinstance(impedance_bus_max, line) else np.zeros((3,3))

        total_impedance['Phase 1'] += imp_dict_max[0][0]
        total_impedance['Phase 2'] += imp_dict_max[1][1]
        total_impedance['Phase 3'] += imp_dict_max[2][2]
        
    return total_impedance
    

In [ ]:
def createRXmatrices(feeder, node_index_map):
    graph = feeder.network
    n = len(graph.nodes) #number of nodes
    R = np.zeros(n, n)
    X = np.zeros(n, n)
    P = {} #initializing line path dictionary
   
    for node in graph.nodes:
        index = node_index_map[node]
        P[index] = get_path_to_substation(feeder, node, node_index_map)
    
    for n_outer in graph.nodes:
        index_outer = node_index_map[n_outer]
        
        for n_inner in graph.nodes:
            index_inner = node_index_map[n_inner]
            intersection_set = set.intersection(set(P[index_outer]), set(P[index_inner]))
            intersection_list = list(intersection_set)
            
            for edge in intersection_list:
                node_name1 = getKey(node_index_map, edge[0])
                node_name2 = getKey(node_index_map, edge[1])
                edge_impedance = get_total_impedance_between_two_buses(feeder, node_name1, node_name2)
            